In [6]:
import numpy as np

import pandas as pd
import os
import glob
import scipy
import gzip
import pickle

import seaborn as sns
import matplotlib.pyplot as plt 

import raycastingfunctions as rc
import detectionfunctions as detection
import importlib
import gc

import time


In [7]:
# common definitions
allnumfish=[10,30,70,151]
cmperpixel = 210/1870  # see 'images/pxpercm-conversion.pdf'


In [8]:

# import
trials=['10-fish/0066/','/10-fish/0105/','/10-fish/0126/','30-fish/0084/','/30-fish/0115/','/30-fish/0120/','70-fish/0103/','70-fish/0107/','70-fish/0124/','150-fish/']
framerate=30
dt=1/framerate

blind_angle = 25/360*np.pi*2  # S LeBlanc used 25 degrees, citing ref:  Diana Pita, Bret A. Moore, Luke P. Tyrrell, Esteban Fernández-Juricic, and Marı́a Ángeles Esteban. Vision in two cyprinid fish: implications for collectivebehavior. PeerJ, 3:e1113, 2015. doi: 10.7717/peerj.1113. URL https://doi.org/10.7717/peerj.1113.
blind_start = np.pi - blind_angle/2
binocularangle = blind_angle/2

datadir = '../data/'
detectiondir = '../detectionresults/'
resultsdir = '../savedresults-final/'

# Process detection results and save condensed form

In [9]:
# import all results together
# processing_skipvalue = 5  # remember, I only kept every 5th one, for the detection calculations


newimport=False
name = 'detection-probabilistic-perfish'
savename = 'detectionresults'
if newimport:
    alldetectionresults=[]
    alldetectionresults_blind=[]
    alldetectionresults_problow=[]
    alldetectionresults_probhigh=[]
    for tnum in range(len(trials)):
        trial=trials[tnum]
        print(trial)
        filedir=detectiondir+trial
        outname = filedir + name+'.pklz'
        [pointdist,allseen,allseen_high,allseen_low,allrelangle,processing_skipvalue] = pickle.load(gzip.open(outname,'rb'))
        # high is high blocking probability of 0.75, low is "low blocking prob" of 0.5
        
        # full blockage / full fieldd
        alldetectionresults.append([pointdist,allseen])    
        
        # blind spot
        allseen_blind = allseen.copy()
        allseen_blind[np.abs(allrelangle)>blind_start] = 0
        alldetectionresults_blind.append([pointdist,allseen_blind])        
        
        # set blind spot for incomplete blocking, and add
        allseen_low[np.abs(allrelangle)>blind_start] = 0        
        allseen_high[np.abs(allrelangle)>blind_start] = 0
        alldetectionresults_problow.append([pointdist,allseen_low])
        alldetectionresults_probhigh.append([pointdist,allseen_high])
        

    # change the data structure to one that is easier to manipulate, by putting the trials together     
    # this is inefficient, I never updated it, but it works
    grid_allseen = [[] for _ in range(4)]
    grid_allseen_problow = [[] for _ in range(4)]
    grid_allseen_probhigh = [[] for _ in range(4)]
    grid_allseen_blind = [[] for _ in range(4)]
    for case in range(4):
        casesel=[range(0,3),range(3,6),range(6,9),[9]]
        for i in range(len(casesel[case])):
            [_,allseen] = alldetectionresults[casesel[case][i]]
            [_,allseen_problow] = alldetectionresults_problow[casesel[case][i]]
            [_,allseen_probhigh] = alldetectionresults_probhigh[casesel[case][i]]
            [_,allseen_blind] = alldetectionresults_blind[casesel[case][i]]
            if i==0:
                grid_allseen[case] = allseen[::processing_skipvalue]
                grid_allseen_problow[case] = allseen_problow[::processing_skipvalue]
                grid_allseen_probhigh[case] = allseen_probhigh[::processing_skipvalue]
                grid_allseen_blind[case] = allseen_blind[::processing_skipvalue]
            else:
                grid_allseen[case] = np.vstack((grid_allseen[case],allseen[::processing_skipvalue]))
                grid_allseen_problow[case] = np.vstack((grid_allseen_problow[case],allseen_problow[::processing_skipvalue]))
                grid_allseen_probhigh[case] = np.vstack((grid_allseen_probhigh[case],allseen_probhigh[::processing_skipvalue]))
                grid_allseen_blind[case] = np.vstack((grid_allseen_blind[case],allseen_blind[::processing_skipvalue]))

             

    del alldetectionresults
    del alldetectionresults_problow
    del alldetectionresults_probhigh
    del alldetectionresults_blind
    gc.collect()
    # now all the results are stored in these arrays
    pickle.dump([grid_allseen, grid_allseen_problow, grid_allseen_probhigh, grid_allseen_blind,processing_skipvalue], gzip.open(resultsdir+savename+'.pklz','wb'))
else:
    [grid_allseen, grid_allseen_problow, grid_allseen_probhigh, grid_allseen_blind,processing_skipvalue] = pickle.load(gzip.open(resultsdir+savename+'.pklz','rb'))        
        
        

## Save just the means, for easy use with the model

In [10]:
mean_toplot = np.array([np.mean(np.mean(x,axis=-1)) for x in grid_allseen_probhigh])
std_toplot = np.array([np.std(np.mean(x,axis=-1)) for x in grid_allseen_probhigh])
groupdetect = [np.sum(x,axis=1) for x in grid_allseen_probhigh]
meannum = np.array([np.mean(x) for x in groupdetect])
stdnum = np.array([np.std(x) for x in groupdetect])
pickle.dump([mean_toplot,std_toplot,meannum,stdnum],open(resultsdir+'ind-groupmeans.pkl','wb'))

# Import data, and also save condensed version

In [11]:
grid_frontbackdist = []
grid_sidesidedist = []
grid_groupstates = []
grid_groupheading = []
grid_orientations = []
grid_tailcoords = []
grid_positions = []
grid_groupcentroid = []
grid_lefteye = []
grid_righteye = []

tnum=0
for tnum in range(len(trials)):
    trial=trials[tnum]
    print('trial: ',trial)
    filedir=datadir+trial

    filenames=np.sort(glob.glob(filedir+'*fov.h5'))

    outname=filedir+'basicdata-v3.pklz'
    [positions,orientations,tailcoords,lefteye,righteye,groupcentroid,groupheading,groupheadingxy,groupstates,pixelwidth,pixelheight,rotcoords] = pickle.load(gzip.open(outname,'rb'))
    frontbackdist=rotcoords[:,:,0]
    sidesidedist=rotcoords[:,:,1] 
    numsteps,numfish, _ = positions.shape
    
    grid_frontbackdist.append(frontbackdist)
    grid_sidesidedist.append(sidesidedist)
    grid_groupstates.append(groupstates)
    grid_groupheading.append(groupheading)
    grid_orientations.append(orientations)
    grid_tailcoords.append(tailcoords)
    grid_positions.append(positions)    
    grid_groupcentroid.append(groupcentroid)
    grid_lefteye.append(lefteye)
    grid_righteye.append(righteye)   

trial:  10-fish/0066/
trial:  /10-fish/0105/
trial:  /10-fish/0126/
trial:  30-fish/0084/
trial:  /30-fish/0115/
trial:  /30-fish/0120/
trial:  70-fish/0103/
trial:  70-fish/0107/
trial:  70-fish/0124/
trial:  150-fish/


In [13]:
datafile = 'data-grid.pklz'
pickle.dump([grid_frontbackdist,grid_sidesidedist,grid_groupstates,grid_groupheading,grid_orientations,grid_tailcoords,grid_positions,grid_groupcentroid,grid_lefteye,grid_righteye],gzip.open(resultsdir+datafile,'wb'))

# Calculate results for example case and save in condensed form

## Regular detection

In [14]:
savefile = 'distributions+exampledata.pkl'
# processing_skipvalue = 5  # remember, I only kept every 5th one, for the detection calculations
exampleframes = np.array([51339,56496,60123,379])
exampleframes_short = np.round(exampleframes/processing_skipvalue).astype(int)  # close enough to the ones I chose originally, should be alright


## Loop through and save the calculations needed in order to make the plot

degreebins = np.linspace(0,1,200)
degreevalues = (degreebins[1:]+degreebins[0:-1])/2
distbins = np.linspace(0,500,120) 
distvalues = (distbins[1:]+distbins[0:-1])/2
distributions1D = np.zeros((4,len(degreebins)-1))
distributions1D_problow = np.zeros((4,len(degreebins)-1))
distributions1D_probhigh = np.zeros((4,len(degreebins)-1))
distributions1D_blind = np.zeros((4,len(degreebins)-1))
exampledata = []

# some functions for convenience, define outside loop
def get1dhist(x):
    return np.histogram(np.reshape(np.mean(x,axis=-1),-1),bins=degreebins, density=True)[0]
def getsmooth1dhist(x):
    kde = scipy.stats.gaussian_kde(np.reshape(np.mean(x,axis=-1),-1))
    return kde.evaluate(degreevalues) 


histfn = getsmooth1dhist
# histfn = get1dhist


for case in range(4):    
    print(case)
    catcasedata = lambda x:  np.concatenate([g[::processing_skipvalue] for g in x[case*3:(case+1)*3]]) # keep inside loop
    # these use processing_skip_value, so can directly compare with detection results
    ssjoined = catcasedata(grid_sidesidedist)
    fbjoined=catcasedata(grid_frontbackdist)
    orientjoined=catcasedata(grid_orientations)
    gcjoined = catcasedata(grid_groupcentroid)
    # treat this one differently, becase the group heading has a length of 1 shorter
    catcasedata_derivativedata = lambda x: np.concatenate([np.insert(g,0,0)[::processing_skipvalue] for g in x[case*3:(case+1)*3]]) # keep inside loop
    groupheadingjoined=catcasedata_derivativedata(grid_groupheading)
    numfish = ssjoined.shape[1]
  
    # save histogram data
    distributions1D[case] = histfn(grid_allseen[case])
    distributions1D_problow[case] = histfn(grid_allseen_problow[case])
    distributions1D_probhigh[case] = histfn(grid_allseen_probhigh[case])
    distributions1D_blind[case] = histfn(grid_allseen_blind[case])
    distance = np.sqrt(np.reshape(ssjoined,-1)**2 + np.reshape(fbjoined,-1)**2)
    
    # example frame data and segments
    step = exampleframes_short[case]
    points = np.array([fbjoined[step,:],ssjoined[step,:]]).T
    orients = orientjoined[step,:]
    # need to rotate around the group heading axis
    xrot = np.cos(groupheadingjoined[step])
    yrot = -np.sin(groupheadingjoined[step])
    rotatefn = lambda x:  np.dot([[xrot,-yrot],[yrot,xrot]],(x-gcjoined[step]).T).T
    le_rot = rotatefn(catcasedata(grid_lefteye)[step])
    re_rot = rotatefn(catcasedata(grid_righteye)[step])
    tail_rot = rotatefn(catcasedata(grid_tailcoords)[step])
    points_tp = rotatefn(catcasedata(grid_positions)[step])
    fsegs = detection.fishmodel(points_tp,le_rot,re_rot,tail_rot)
#     segments = np.reshape(fsegs,(-1,2,2))
    seen = np.mean(grid_allseen[case][step],axis=-1)
    seen_problow = np.mean(grid_allseen_problow[case][step],axis=-1)
    seen_probhigh = np.mean(grid_allseen_probhigh[case][step],axis=-1)
    seen_blind = np.mean(grid_allseen_blind[case][step],axis=-1)
    exampledata.append([fsegs,points,orients-groupheadingjoined[step],le_rot,re_rot,tail_rot,seen,seen_problow,seen_probhigh,seen_blind])
    

d1D = [distributions1D,distributions1D_problow,distributions1D_probhigh,distributions1D_blind]
pickle.dump([degreebins,distbins,d1D,exampledata], open(resultsdir+savefile,'wb'))


0
1
2
3
